In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI,GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document
from langchain_astradb import AstraDBVectorStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv
import pandas as pd
import os

In [38]:
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
llm = ChatGoogleGenerativeAI(model = "gemini-1.5-pro",temperature = 0.8)

print(llm.invoke("What is capital of India").content)

The capital of India is **New Delhi**. 



In [39]:
symptoms_data = pd.read_csv(r"C:\Users\aashutosh kumar\projects\Ayurvedic-Remedy-Bot\data\ayurvedic_symptoms_desc.csv")
symptoms_data.head(4)

,Symptom,Description
0,Vatavikara,Disorders related to the Vata dosha.
1,Netraroga,Eye disorders.
2,Malavarodha,Constipation.
3,Sutikadosha,Postpartum disorders.


In [40]:
medication_data = pd.read_csv(r"C:\Users\aashutosh kumar\projects\Ayurvedic-Remedy-Bot\data\Formulation-Indications.csv")
medication_data.head(4)

,Name of Medicine,Reference text,Dispensing Pack Size,Main Indications,Dose,Precaution/ Contraindication,Preferred use (OPD/ IPD),Class
0,Abhayarishta,AFI,200 ml,"Arsha, Agnimandya,\nUdararoga, Vibandha",12 - 24 ml,NS,Both,A
1,Amritarishta,AFI,200 ml,"SarvaJvara, Jirna Jvara",12 - 24 ml,NS,Both,A
2,Aragvadharishta,AH,200ml,"Kandu, Tvak Vikara,\nVibandha",12 - 24 ml,NS,Both,A
3,Aravindasava,AFI,200 ml,"Balaroga, Balakshaya,\nAgnimandya, Aruchi",12 - 24 ml,NS,Both,A


In [41]:
class_labels = pd.read_csv(r"data\FormulationClass.csv")
class_labels.head(3)

,Class,Unnamed: 1
0,A,Asava Arista
1,B,Arka Kalpana
2,C,"Avaleha, Khand, Pak Kalpana"


In [42]:
classes = dict(zip(class_labels["Class"],class_labels["Unnamed: 1"]))
medication_data["Class"] = medication_data["Class"].map(classes)

In [44]:
medication_data.head()

,Name of Medicine,Reference text,Dispensing Pack Size,Main Indications,Dose,Precaution/ Contraindication,Preferred use (OPD/ IPD),Class
0,Abhayarishta,AFI,200 ml,"Arsha, Agnimandya,\nUdararoga, Vibandha",12 - 24 ml,NS,Both,Asava Arista
1,Amritarishta,AFI,200 ml,"SarvaJvara, Jirna Jvara",12 - 24 ml,NS,Both,Asava Arista
2,Aragvadharishta,AH,200ml,"Kandu, Tvak Vikara,\nVibandha",12 - 24 ml,NS,Both,Asava Arista
3,Aravindasava,AFI,200 ml,"Balaroga, Balakshaya,\nAgnimandya, Aruchi",12 - 24 ml,NS,Both,Asava Arista
4,Arjunarishta/ Parthadyarishta,AFI,200 ml,"Hridroga, Hriddrava, Hrid- daurbalya, Moha,\nM...",12 - 24 ml,NS,Both,Asava Arista


In [43]:
def csv_to_document(data):
    list = []

    for i,row in data.iterrows():
        dict = {
            "product_name" : row["product_title"],
            "review" : row["review"]
        }
        product_list.append(dict)

In [45]:
medication_data.head(2)

,Name of Medicine,Reference text,Dispensing Pack Size,Main Indications,Dose,Precaution/ Contraindication,Preferred use (OPD/ IPD),Class
0,Abhayarishta,AFI,200 ml,"Arsha, Agnimandya,\nUdararoga, Vibandha",12 - 24 ml,NS,Both,Asava Arista
1,Amritarishta,AFI,200 ml,"SarvaJvara, Jirna Jvara",12 - 24 ml,NS,Both,Asava Arista


**PREPARING THE DATASET OF SYMPTOMS AND DESC**

In [50]:
symptom_list = []
for i,row in symptoms_data.iterrows():
    dict = {
        "Symptom" : row["Symptom"],
        "Description" : row["Description"] 
    }
    symptom_list.append(dict)

symptom_list[:6]

[{'Symptom': 'Vatavikara',
  'Description': 'Disorders related to the Vata dosha.'},
 {'Symptom': 'Netraroga', 'Description': 'Eye disorders.'},
 {'Symptom': 'Malavarodha', 'Description': 'Constipation.'},
 {'Symptom': 'Sutikadosha', 'Description': 'Postpartum disorders.'},
 {'Symptom': 'Vrana', 'Description': 'Wounds or injuries.'},
 {'Symptom': 'Khalitya', 'Description': 'Hair loss.'}]

In [55]:
symp_docs = []
for dict in symptom_list:
    metadata = {"Symptom" : dict["Symptom"]}
    doc = Document(page_content = dict["Description"],metadata = metadata)
    symp_docs.append(doc)
symp_docs[:5]

[Document(metadata={'Symptom': 'Vatavikara'}, page_content='Disorders related to the Vata dosha.'),
 Document(metadata={'Symptom': 'Netraroga'}, page_content='Eye disorders.'),
 Document(metadata={'Symptom': 'Malavarodha'}, page_content='Constipation.'),
 Document(metadata={'Symptom': 'Sutikadosha'}, page_content='Postpartum disorders.'),
 Document(metadata={'Symptom': 'Vrana'}, page_content='Wounds or injuries.')]

**CONVERTING MEDICATIONS DATFRAME TO DOCUMENT**

In [58]:
medicines_list = []

for i, row in medication_data.iterrows():
    med_dict = {
        "Name of Medicine": row["Name of Medicine"],
        "Reference text": row["Reference text"],
        "Dispensing Pack Size": row["Dispensing Pack Size"],
        "Main Indications": row["Main Indications"],
        "Dose": row["Dose"],
        "Precaution/ Contraindication": row["Precaution/ Contraindication"],
        "Preferred use (OPD/ IPD)": row["Preferred use (OPD/ IPD)"],
        "Class": row["Class"]
    }
    medicines_list.append(med_dict)

medicines_list[:2]

[{'Name of Medicine': 'Abhayarishta',
  'Reference text': 'AFI',
  'Dispensing Pack Size': '200 ml',
  'Main Indications': 'Arsha, Agnimandya,\nUdararoga, Vibandha',
  'Dose': '12 - 24 ml',
  'Precaution/ Contraindication': 'NS',
  'Preferred use (OPD/ IPD)': 'Both',
  'Class': 'Asava Arista '},
 {'Name of Medicine': 'Amritarishta',
  'Reference text': 'AFI',
  'Dispensing Pack Size': '200 ml',
  'Main Indications': 'SarvaJvara, Jirna Jvara',
  'Dose': '12 - 24 ml',
  'Precaution/ Contraindication': 'NS',
  'Preferred use (OPD/ IPD)': 'Both',
  'Class': 'Asava Arista '}]

In [59]:
med_docs = []

for med_dict in medicines_list:
    metadata = {
        "Name of Medicine": med_dict["Name of Medicine"],
        "Reference text": med_dict["Reference text"],
        "Dispensing Pack Size": med_dict["Dispensing Pack Size"],
        "Main Indications": med_dict["Main Indications"],
        "Dose": med_dict["Dose"],
        "Precaution/ Contraindication": med_dict["Precaution/ Contraindication"],
        "Preferred use (OPD/ IPD)": med_dict["Preferred use (OPD/ IPD)"],
        "Class": med_dict["Class"]
    }
    doc = Document(page_content=f"Name of Medicine: {med_dict['Name of Medicine']}\n"
                                f"Reference text: {med_dict['Reference text']}\n"
                                f"Dispensing Pack Size: {med_dict['Dispensing Pack Size']}\n"
                                f"Main Indications: {med_dict['Main Indications']}\n"
                                f"Dose: {med_dict['Dose']}\n"
                                f"Precaution/ Contraindication: {med_dict['Precaution/ Contraindication']}\n"
                                f"Preferred use (OPD/ IPD): {med_dict['Preferred use (OPD/ IPD)']}\n"
                                f"Class: {med_dict['Class']}",
                      metadata=metadata)
    med_docs.append(doc)

med_docs[:5]


[Document(metadata={'Name of Medicine': 'Abhayarishta', 'Reference text': 'AFI', 'Dispensing Pack Size': '200 ml', 'Main Indications': 'Arsha, Agnimandya,\nUdararoga, Vibandha', 'Dose': '12 - 24 ml', 'Precaution/ Contraindication': 'NS', 'Preferred use (OPD/ IPD)': 'Both', 'Class': 'Asava Arista '}, page_content='Name of Medicine: Abhayarishta\nReference text: AFI\nDispensing Pack Size: 200 ml\nMain Indications: Arsha, Agnimandya,\nUdararoga, Vibandha\nDose: 12 - 24 ml\nPrecaution/ Contraindication: NS\nPreferred use (OPD/ IPD): Both\nClass: Asava Arista '),
 Document(metadata={'Name of Medicine': 'Amritarishta', 'Reference text': 'AFI', 'Dispensing Pack Size': '200 ml', 'Main Indications': 'SarvaJvara, Jirna Jvara', 'Dose': '12 - 24 ml', 'Precaution/ Contraindication': 'NS', 'Preferred use (OPD/ IPD)': 'Both', 'Class': 'Asava Arista '}, page_content='Name of Medicine: Amritarishta\nReference text: AFI\nDispensing Pack Size: 200 ml\nMain Indications: SarvaJvara, Jirna Jvara\nDose: 12 -

**STORING THE DOCUMENTS TO VECTORSTORE**

In [65]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x0000022AE4846F50>, model='models/embedding-001', task_type=None, google_api_key=None, credentials=None, client_options=None, transport=None, request_options=None)

In [72]:
load_dotenv()
ASTRA_API_KEY = os.getenv("ASTRA_API")
DB_ENDPOINT = os.getenv("DB_ENDPOINT")
DB_ID = os.getenv("DB_ID")

In [73]:
vector_store = AstraDBVectorStore(collection_name = "Ayurvedic_Bot",
                               embedding = embeddings,
                               api_endpoint = DB_ENDPOINT,
                               token = ASTRA_API_KEY,
                               namespace = "symptoms")
    
inserted_ids = vector_store.add_documents(symp_docs)
print(f"Inserted {len(inserted_ids)} documents")



Inserted 300 documents


In [85]:
results = vector_store.similarity_search("I have headache")
symptom_names = []

for res in results:
    symptom_name = res.metadata.get("Symptom", "Unknown")
    # print(f"* {res.page_content} [{res.metadata}]")
    symptom_names.append(symptom_name)

print(symptom_names[0])

Pittajashirahshula
